# Getting the raw dataset

We'll be using `twython` and `cryptocompare` as APIs to get the respective
Twitter and cryptocurrency data that will make up the project's dataset.

## Importing packages.

First, we'll import the necessary python packages.


In [1]:
from datetime import datetime, timedelta

# Access environment variables.
from os import environ

# Resolving paths in a platform agnostic way.
from os.path import dirname, join, realpath

# Cryptocompare API.
from cryptocompare import get_historical_price_minute, get_price

# Loading environment variables from a `.env` file.
from dotenv import load_dotenv

# Manipulating the raw data to save it in a ``.csv`` files.
from pandas import DataFrame, DatetimeIndex
from pandas import concat as concat_df
from pandas import date_range

# Twython API.
from twython import Twython

In [2]:
def is_interactive():
    """Check if the script is being run interactively."""
    import __main__ as main

    return not hasattr(main, "__file__")


if is_interactive():
    SCRIPT_DIR = dirname(realpath("__file__"))
else:
    SCRIPT_DIR = dirname(realpath(__file__))

# "../data"
DATA_DIR = join(dirname(SCRIPT_DIR), "data")

## Initializing APIs.

Next, we need to set credentials to access the CryptoCompare and Twython
APIs. To avoid hard coding the secrets, we'll load them from the environment,
i.e., a `.env` file.

In [3]:
# Loads environment variables from a `.env` file.
load_dotenv()

# Now the environment variables from the file are available, as if they were
# specified typically from the commandline.
TWITTER_APP_KEY = environ["TWITTER_APP_KEY"]
TWITTER_APP_SECRET = environ["TWITTER_APP_SECRET"]

twitter = Twython(TWITTER_APP_KEY, TWITTER_APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(
    TWITTER_APP_KEY, TWITTER_APP_SECRET, access_token=ACCESS_TOKEN
)

## Testing the APIs.

Let's test the CryptoCompare API. `CRYPTOCOMPARE_API_KEY` should be specified
in the `.env` file so that the python package can detect it automatically as
an environment variable.

In [4]:
get_price("BTC", currency="USD")

{'BTC': {'USD': 41507.94}}

Now let's test accessing Twitter's API through Twython.

In [5]:
twitter.search(count=1, q="cryptocurrency")

{'statuses': [{'created_at': 'Fri Mar 18 17:22:15 +0000 2022',
   'id': 1504870812020776962,
   'id_str': '1504870812020776962',
   'text': 'RT @chitaglorya__: giveaway 200$ | 2,800,000 IDR\n\n-- rt &amp; follow @flagnetwork_NFT \n\n-- rt &amp; quoted their pinned with   #FlagNetwork  #crypt…',
   'truncated': False,
   'entities': {'hashtags': [{'text': 'FlagNetwork', 'indices': [127, 139]}],
    'symbols': [],
    'user_mentions': [{'screen_name': 'chitaglorya__',
      'name': 'chita glorya🍒',
      'id': 1449414811436023812,
      'id_str': '1449414811436023812',
      'indices': [3, 17]},
     {'screen_name': 'flagnetwork_NFT',
      'name': 'Flag Network',
      'id': 1055135887233695744,
      'id_str': '1055135887233695744',
      'indices': [69, 85]}],
    'urls': []},
   'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
   'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
   'in_reply_to_status_id': None,
   'in_re

## Getting cryptocurrency data

In [7]:
CRYPTOCURRENCIES = ["BTC", "ETH", "DOGE", "SOL", "AVAX"]
# The last 7 days is the limit of the minute data from crypto compare.
NUM_DAYS = 7
# Decided based on limitations of API at the time of data collection.
LAST_DAY = datetime.today()
DATE_RANGE = date_range(end=LAST_DAY, periods=NUM_DAYS)


def get_and_save_crypto_dataset(
    cryptocurrencies: list[str], time_period: DatetimeIndex, save_folder: str
):
    """Get cryptocurrency data and save it to a csv file."""
    for cryptocurrency in cryptocurrencies:
        price_dataset = []
        for date in time_period:
            price_dataset += get_historical_price_minute(
                cryptocurrency, "USD", limit=1440, toTs=date
            )

        # Saving the raw price data to a csv file.
        price_data_frame = DataFrame(price_dataset)
        price_data_frame.to_csv(
            join(
                save_folder,
                "raw",
                "crypto",
                f"{cryptocurrency.lower()}"
                f"_{(time_period[0] - timedelta(days=1)).strftime('%Y_%m_%d')}"
                f"-{time_period[-1].strftime('%Y_%m_%d')}_minute.csv",
            )
        )


get_and_save_crypto_dataset(CRYPTOCURRENCIES, DATE_RANGE, DATA_DIR)

## Getting Twitter data

In [16]:
tweet_data: dict = {
    "text": [],  # statuses
    "retweet_count": [],
    "favorite_count": [],
    "followers_count": [],
    "verified": [],
    "listed_count": [],
    "created_at": [],
    "hashtags": [],
    "name": [],
}

HASHTAG_LIST = [
    "#cryptocurrency",
    "#crypto",
    "#dogecoin",
    "#DOGE",
    "#bitcoin",
    "#BTC",
    "#ethereum",
    "#ENS",
    "#avalanche",
    "#AVAX",
    "#solana",
    "#SOL",
]


def read_tweets(search_results):
    """Read tweets from the search results into a data frame."""
    for i in range(len(search_results["statuses"])):
        for j, _ in tweet_data.items():
            if (
                (j == "text")
                or (j == "retweet_count")
                or (j == "favorite_count")
            ):
                tweet_data[j].append(search_results["statuses"][i][j])
            elif j == "hashtags":
                tweet_data[j].append(
                    search_results["statuses"][i]["entities"][j]
                )
            else:
                tweet_data[j].append(search_results["statuses"][i]["user"][j])

    return DataFrame(tweet_data)


date_today = datetime.today().strftime("%Y-%m-%d")


def make_df(hashtag_list, until_date=date_today, result_type="popular"):
    """Make a dataframe of tweets containing the specified hashtags."""
    count = 0
    tweets_dataframe = DataFrame()
    for i in hashtag_list:
        search_results = twitter.search(
            count=100, q=i, until=until_date, result_type=result_type
        )
        tweets_dataframe = concat_df(
            [tweets_dataframe, read_tweets(search_results)]
        )
        count = count + 1

    return tweets_dataframe


df = make_df(HASHTAG_LIST)

df.to_csv(
    join(
        DATA_DIR,
        "raw",
        "twitter",
        f"tweets_{date_today}.csv".replace("-", "_"),
    )
)

df.head()

,text,retweet_count,favorite_count,followers_count,verified,listed_count,created_at,hashtags,name
0,We are ready...\nAre you? \n\nThe clock is tic...,913,2415,394003,True,568,Mon Feb 01 22:37:17 +0000 2021,[],FEG (Feed Every Gorilla)
1,No swindling here 🚀 😉 TinderSwindler Coin is l...,446,469,1334485,True,600,Fri Sep 17 13:25:03 +0000 2010,[],Candela
2,JUST IN: President Joe Biden to sign an execut...,735,4560,701689,True,1419,Wed Apr 28 20:04:12 +0000 2021,"[{'text': 'Bitcoin', 'indices': [77, 85]}, {'t...",Watcher.Guru
3,#catecoin🐱 app will support multiple languages...,364,1053,82238,True,170,Sat Mar 30 04:19:58 +0000 2019,"[{'text': 'catecoin', 'indices': [0, 9]}]",CateCoin
4,The most undervalued #cryptocurrency is…… 👇🏼,366,2174,610801,True,2271,Fri Jul 23 20:33:57 +0000 2010,"[{'text': 'cryptocurrency', 'indices': [21, 36]}]",David Gokhshtein
